In [ ]:
!sudo apt update
!sudo apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install git+https://github.com/ozgur/python-firebase
!pip install -U pyarrow
!pip install  pyspark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [931 kB]
Get:13 http://archive.

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, json_tuple,to_json,from_json
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from firebase import firebase
from datetime import datetime
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.ml.feature import VectorAssembler
from google.colab import  drive
import pandas as pd
import json
import time

In [ ]:
sc = SparkContext("local[*]", "app")

In [ ]:
!pip install git+https://github.com/ozgur/python-firebase
from firebase import firebase
url = 'https://iotapp-ae6e4-default-rtdb.firebaseio.com/'
firebase = firebase.FirebaseApplication(url)

  Cloning https://github.com/ozgur/python-firebase to /tmp/pip-req-build-hb_9plw2
  Running command git clone -q https://github.com/ozgur/python-firebase /tmp/pip-req-build-hb_9plw2


In [ ]:
def Average(lst):
    return sum(lst) / len(lst)

In [ ]:
drive.mount('/drive')
k=0
while True:
  result = firebase.get('', '')
  listKeys=list(result["User"].keys())
  i=0
  for user in listKeys:
    result["User"]['data'+str(i)]= result["User"].pop(user)
    i+=1
  with open('/drive/My Drive/data.json', 'w') as mon_fichier:
    json.dump(result['User'], mon_fichier)
  spark = SparkSession(sc)
  jsonDF = spark.read.option("multiline","true").json("/drive/My Drive/data.json")
  df = []
  for user in jsonDF.columns:
    df1 =jsonDF.select(user)
    df.append(df1)
  df_complete = reduce(DataFrame.union, df)
  DF = df_complete.select('data0.*')
  DF.show()
  DataF =DF.toPandas()
  Temp=DataF.Température[k:k+15].astype(int)
  Tens=DataF.Tension[k:k+15].astype(int)
  Freq=list(DataF[['frequence cardiaque']][k:k+15].astype(int).values)
  data={'Température': Average(Temp),'Tension':Average(Tens), 'frequence cardiaque':Average(Freq)[0]}
  result = firebase.post("AVERAGE", data)
  k+=15
  print("Send at : %s" % time.ctime())
  time.sleep(900)

Mounted at /drive
+-----------+-------+-------------------+
|Température|Tension|frequence cardiaque|
+-----------+-------+-------------------+
|         35|     13|                 55|
|         37|     11|                 76|
|         38|     14|                 78|
|         37|     10|                 81|
|         35|     10|                 65|
|         36|     12|                 62|
|         37|     10|                 81|
|         38|     11|                 68|
|         35|     10|                 75|
|         36|     13|                 74|
|         37|     11|                 63|
|         37|     12|                 62|
|         38|     10|                 80|
|         36|     13|                 62|
|         36|     10|                 61|
|         35|     12|                 56|
|         38|     13|                 60|
|         38|     13|                 69|
|         36|     14|                 72|
|         38|     12|                 66|
+-----------+---

KeyboardInterrupt: ignored